In [1]:
import numpy as np
import pandas as pd

In [2]:
dftrain = pd.read_csv('titanic/train.csv')
dftest = pd.read_csv('titanic/test.csv')

In [3]:
display(dftrain.head())
display(dftest.head())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
display(dftrain.isnull().sum())
display(dftest.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [5]:
dftrain['Age']=dftrain['Age'].fillna(dftrain['Age'].median())
dftrain['Embarked']=dftrain['Embarked'].fillna('S')

dftest['Age']= dftest['Age'].fillna(dftest['Age'].median())
dftest['Fare']= dftest['Fare'].fillna(dftest['Fare'].median())

In [6]:
dftrain.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [7]:
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X = dftrain[features]
y = dftrain['Survived'].values
X_sub = dftest[features]

In [8]:
X = pd.get_dummies(X)
X_sub = pd.get_dummies(X_sub)

In [9]:
display(X.info())
display(X_sub.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      891 non-null    int64  
 1   Age         891 non-null    float64
 2   SibSp       891 non-null    int64  
 3   Parch       891 non-null    int64  
 4   Fare        891 non-null    float64
 5   Sex_female  891 non-null    uint8  
 6   Sex_male    891 non-null    uint8  
 7   Embarked_C  891 non-null    uint8  
 8   Embarked_Q  891 non-null    uint8  
 9   Embarked_S  891 non-null    uint8  
dtypes: float64(2), int64(3), uint8(5)
memory usage: 39.3 KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      418 non-null    int64  
 1   Age         418 non-null    float64
 2   SibSp       418 non-null    int64  
 3   Parch       418 non-null    int64  
 4   Fare        418 non-null    float64
 5   Sex_female  418 non-null    uint8  
 6   Sex_male    418 non-null    uint8  
 7   Embarked_C  418 non-null    uint8  
 8   Embarked_Q  418 non-null    uint8  
 9   Embarked_S  418 non-null    uint8  
dtypes: float64(2), int64(3), uint8(5)
memory usage: 18.5 KB


None

Knn: 0.77272, 0.76076

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

steps = [('scaler', StandardScaler()),
         ('knn', KNeighborsClassifier())]

pipeline = Pipeline(steps)

parameters = {'knn__n_neighbors':np.arange(1,50)}

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 21)

cv = GridSearchCV(pipeline, parameters, cv=3)
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)
y_sub = cv.predict(X_sub)
print("Accuracy: {}".format(cv.score(X_test, y_test)))
print("Accuracy: {}".format(cv.score(X_train, y_train)))
print("Tuned Model Parameters: {}".format(cv.best_params_))

Accuracy: 0.8156424581005587
Accuracy: 0.8384831460674157
Tuned Model Parameters: {'knn__n_neighbors': 12}


In [11]:
predictions = pd.DataFrame(y_sub)
gender_submission = pd.read_csv('titanic/gender_submission.csv')
datasets = pd.concat(([gender_submission['PassengerId'],predictions]),axis=1)
# creating the submission file
datasets.columns=['PassengerId','Survived']
datasets.to_csv('titanic/gender_submission.csv',index=False) 

SVC 0.78468

In [12]:
from sklearn.svm import SVC

steps = [('scaler', StandardScaler()),
         ('SVM', SVC())]

parameters = {'SVM__C':np.linspace(0, 100, 21)[1:],
              'SVM__gamma':np.linspace(0.001, 0.01, 10)}

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)

pipeline = Pipeline(steps)
cv = GridSearchCV(pipeline, parameters, cv=3)
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

y_sub = cv.predict(X_sub)
print("Accuracy: {}".format(cv.score(X_test, y_test)))
print("Accuracy: {}".format(cv.score(X_train, y_train)))
print("Tuned Model Parameters: {}".format(cv.best_params_))

Accuracy: 0.8100558659217877
Accuracy: 0.8441011235955056
Tuned Model Parameters: {'SVM__C': 50.0, 'SVM__gamma': 0.01}


In [13]:
predictions = pd.DataFrame(y_sub)
gender_submission = pd.read_csv('titanic/gender_submission.csv')
datasets = pd.concat(([gender_submission['PassengerId'],predictions]),axis=1)
# creating the submission file
datasets.columns=['PassengerId','Survived']
datasets.to_csv('titanic/gender_submission.csv',index=False) 

LogisticRegression

In [15]:
from sklearn.linear_model import LogisticRegression

steps = [('scaler', StandardScaler()),
         ('logre', LogisticRegression(max_iter=1000, solver='liblinear'))]
c_space = np.logspace(-5, 8, 15) # chọn 15 mẫu C từ 10^-5 đến 10^8
parameters = {'logre__C': c_space, 'logre__penalty' : ['l1', 'l2']}


X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)

pipeline = Pipeline(steps)
cv = GridSearchCV(pipeline, parameters, cv=3)
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

y_sub = cv.predict(X_sub)
print("Accuracy: {}".format(cv.score(X_test, y_test)))
print("Accuracy: {}".format(cv.score(X_train, y_train)))
print("Tuned Model Parameters: {}".format(cv.best_params_))

Accuracy: 0.8100558659217877
Accuracy: 0.800561797752809
Tuned Model Parameters: {'logre__C': 3.727593720314938, 'logre__penalty': 'l2'}


In [16]:
predictions = pd.DataFrame(y_sub)
gender_submission = pd.read_csv('titanic/gender_submission.csv')
datasets = pd.concat(([gender_submission['PassengerId'],predictions]),axis=1)
# creating the submission file
datasets.columns=['PassengerId','Survived']
datasets.to_csv('titanic/gender_submission.csv',index=False) 

DecisionTreeClassifier 0.77511

In [20]:
from sklearn.tree import DecisionTreeClassifier

steps = [('scaler', StandardScaler()),
         ('tree', DecisionTreeClassifier())]
parameters = {"tree__max_depth": [3, None], "tree__criterion": ["gini", "entropy"]}

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 42)

pipeline = Pipeline(steps)
cv = GridSearchCV(pipeline, parameters, cv=3)
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

y_sub = cv.predict(X_sub)
print("Accuracy: {}".format(cv.score(X_test, y_test)))
print("Accuracy: {}".format(cv.score(X_train, y_train)))
print("Tuned Model Parameters: {}".format(cv.best_params_))

Accuracy: 0.8171641791044776
Accuracy: 0.8314606741573034
Tuned Model Parameters: {'tree__criterion': 'entropy', 'tree__max_depth': 3}


In [21]:
predictions = pd.DataFrame(y_sub)
gender_submission = pd.read_csv('titanic/gender_submission.csv')
datasets = pd.concat(([gender_submission['PassengerId'],predictions]),axis=1)
# creating the submission file
datasets.columns=['PassengerId','Survived']
datasets.to_csv('titanic/gender_submission.csv',index=False) 